In [1]:
cd /home/linuxmint/data-ops-assignment

/home/linuxmint/data-ops-assignment


In [2]:
from mock_aws import mock_aws

s3_files, dynamodb_items = mock_aws()
print(dynamodb_items[0])

ModuleNotFoundError: No module named 'mock_aws'

In [ ]:
import os

from pandas import DataFrame, read_json, read_csv, read_excel

def load(path: str) -> DataFrame:
    path_split = os.path.split(path)
    if path_split[1].endswith('.json'):
        return read_json(path)
    elif path_split[1].endswith('.csv'):
        return read_csv(path)
    elif path_split[1].endswith('.xlsx'):
        return read_excel(path)

In [ ]:
defects = load('/home/linuxmint/data-ops-assignment/data/defects/defects.xlsx')
print(len(defects.index))
defects.head()

283


,product_id,part_id,buyer_id
0,f0fc8d82-74c8-483e-97e6-20c776836db3,3.520921e+09,1038.0
1,f9d8afda-e632-48b0-a9c1-230934f778e0,7.562914e+09,1368.0
2,e9d9a509-ba18-452f-9411-b5b2595cc82a,3.685856e+09,1878.0
3,f580d855-235d-4c6d-ac6c-7f9f6ef6504c,1.451072e+09,326.0
4,0fcac216-9551-4c79-b4fd-a307879f5c24,1.957686e+09,329.0


In [ ]:
from pandas import notna

users = DataFrame.from_dict(dynamodb_items)
users['last_name_present'] = notna(users['last_name'])
users['lafost_name_present'] = notna(users['lafost_name'])
print(f'Rows with last name missing: {len(users.loc[users["last_name_present"] == False].index)}')
print(f'Rows with lafost name present: {len(users.loc[users["lafost_name_present"] == True].index)}')
missing_last_name = users.loc[users['last_name_present'] == False]
missing_last_name.head()

Rows with last name missing: 1
Rows with lafost name present: 1


,id,first_name,last_name,email,city,age,gender,part_color,lafost_name,last_name_present,lafost_name_present
7,8,Bailie,NaN,bmellody7@flickr.com,Dinalongan,21,None,Orange,Mellody,False,True


In [ ]:
users.loc[users['last_name_present'] == False, 'last_name'] = users['lafost_name']
users['last_name_present'] = notna(users['last_name'])
print(f'Rows with last name missing: {len(users.loc[users["last_name_present"] == False].index)}')
users = users.drop(columns=['lafost_name_present', 'last_name_present', 'lafost_name'])
users.head()

Rows with last name missing: 0


,id,first_name,last_name,email,city,age,gender,part_color
0,1,Alonso,Jarvis,ajarvis0@bizjournals.com,Capacho Nuevo,None,M,Teal
1,2,Montague,Barthrop,mbarthrop1@google.pl,Bojawa,None,M,Orange
2,3,Daile,Boggon,dboggon2@mediafire.com,Chaloem Phra Kiat,58,None,Turquoise
3,4,Domingo,Aizikovitz,daizikovitz3@sina.com.cn,Huochezhan,31,M,Yellow
4,5,Bidget,Daice,bdaice4@nytimes.com,Dongmazar,81,F,Green


In [ ]:
import io

s3_dataframes =  [read_csv(io.BytesIO(f.read())) for f in s3_files]

from pandas import concat

plants = concat(s3_dataframes)
print(len(plants.index))
plants.head()

5000


,id,production_date,car_model,car_color,part_id,part_color,part_material,plant_location,temperature,humidity,pressure
0,99a38e48-a9bc-4348-af0a-0cd19ae6f11f,1/15/2021,Intrepid,Indigo,1877853410,#a7eeff,Brass,China,10.3,97.87,1.7144
1,fb8c1eff-30f3-442d-9032-0f75314d4383,1/25/2021,Xtra,Green,5937437683,#1a832a,Stone,China,9.7,51.54,1.2491
2,f18b75e8-60c1-4cd1-9890-0e46e0b472a7,1/7/2021,Intrigue,Yellow,1036353117,#1f4423,Brass,China,6.6,17.08,1.6865
3,d2d6fb8d-287c-4931-a593-2bff6c899c04,1/12/2021,ES,Violet,5459482372,#bcf42d,Brass,China,105.6,50.99,1.0881
4,8a0553fe-bf9f-428a-868e-3a6124fa2831,1/4/2021,Yukon,Puce,9983813262,#9909fe,Plastic,China,77.5,66.51,1.8042


In [ ]:
from pandas import merge

users_defects = merge(left=defects, right=users, how='left', left_on='buyer_id', right_on='id')
users_defects = users_defects.drop(columns='id')
users_defects.head()

,product_id,part_id,buyer_id,first_name,last_name,email,city,age,gender,part_color
0,f0fc8d82-74c8-483e-97e6-20c776836db3,3.520921e+09,1038.0,Dew,Sor,dsor11@slate.com,Mianhu,29,None,Fuscia
1,f9d8afda-e632-48b0-a9c1-230934f778e0,7.562914e+09,1368.0,Tedd,Eede,teedea7@csmonitor.com,Zhichanghe,38,M,Maroon
2,e9d9a509-ba18-452f-9411-b5b2595cc82a,3.685856e+09,1878.0,Buddie,Empleton,bempletonod@phoca.cz,Xuân Trường,53,M,Turquoise
3,f580d855-235d-4c6d-ac6c-7f9f6ef6504c,1.451072e+09,326.0,Bernardina,Skip,bskip91@webmd.com,Bar-le-Duc,None,F,Yellow
4,0fcac216-9551-4c79-b4fd-a307879f5c24,1.957686e+09,329.0,Rachelle,Boffin,rboffin94@examiner.com,Vancouver,46,F,Goldenrod


In [ ]:
df = merge(left=plants, right=users_defects, how='outer', left_on=['id', 'part_id'], right_on=['product_id', 'part_id'])
df.head()

,id,production_date,car_model,car_color,part_id,part_color_x,part_material,plant_location,temperature,humidity,pressure,product_id,buyer_id,first_name,last_name,email,city,age,gender,part_color_y
0,99a38e48-a9bc-4348-af0a-0cd19ae6f11f,1/15/2021,Intrepid,Indigo,1877853410,#a7eeff,Brass,China,10.3,97.87,1.7144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fb8c1eff-30f3-442d-9032-0f75314d4383,1/25/2021,Xtra,Green,5937437683,#1a832a,Stone,China,9.7,51.54,1.2491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f18b75e8-60c1-4cd1-9890-0e46e0b472a7,1/7/2021,Intrigue,Yellow,1036353117,#1f4423,Brass,China,6.6,17.08,1.6865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,d2d6fb8d-287c-4931-a593-2bff6c899c04,1/12/2021,ES,Violet,5459482372,#bcf42d,Brass,China,105.6,50.99,1.0881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8a0553fe-bf9f-428a-868e-3a6124fa2831,1/4/2021,Yukon,Puce,9983813262,#9909fe,Plastic,China,77.5,66.51,1.8042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(f'Rows with id is null: {df["id"].isnull().sum()}')
print(f'Rows with id is null: {df["product_id"].isnull().sum()}')
print(len(df.index))

Rows with id is null: 0
Rows with id is null: 4717
5000


In [ ]:
df['defect'] = df['product_id'].notnull()
df = df.drop(columns=['product_id', 'part_color_y']) # part_color_y is from user data which implies that all parts bought by the same user have the same color -> wierd
df = df.rename(columns={'id': 'product_id', 'part_color_x': 'part_color'})
print(df.columns)
df.head()

Index(['product_id', 'production_date', 'car_model', 'car_color', 'part_id',
       'part_color', 'part_material', 'plant_location', 'temperature',
       'humidity', 'pressure', 'buyer_id', 'first_name', 'last_name', 'email',
       'city', 'age', 'gender', 'defect'],
      dtype='object')


,product_id,production_date,car_model,car_color,part_id,part_color,part_material,plant_location,temperature,humidity,pressure,buyer_id,first_name,last_name,email,city,age,gender,defect
0,99a38e48-a9bc-4348-af0a-0cd19ae6f11f,1/15/2021,Intrepid,Indigo,1877853410,#a7eeff,Brass,China,10.3,97.87,1.7144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,fb8c1eff-30f3-442d-9032-0f75314d4383,1/25/2021,Xtra,Green,5937437683,#1a832a,Stone,China,9.7,51.54,1.2491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,f18b75e8-60c1-4cd1-9890-0e46e0b472a7,1/7/2021,Intrigue,Yellow,1036353117,#1f4423,Brass,China,6.6,17.08,1.6865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,d2d6fb8d-287c-4931-a593-2bff6c899c04,1/12/2021,ES,Violet,5459482372,#bcf42d,Brass,China,105.6,50.99,1.0881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,8a0553fe-bf9f-428a-868e-3a6124fa2831,1/4/2021,Yukon,Puce,9983813262,#9909fe,Plastic,China,77.5,66.51,1.8042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
df_defect = df.loc[df['defect'] == True]
df_working = df.loc[df['defect'] == False]

for column in ['product_id', 'production_date', 'car_model', 'car_color', 'part_id', 'part_color', 'part_material', 'plant_location']:
    print(f'________{column}_______')
    print('ALL' + os.linesep + str(df[column].value_counts()))
    print('DEFECT' + os.linesep + str(df_defect[column].value_counts()))
    print('WORKING' + os.linesep + str(df_working[column].value_counts()))

# product_id, part_id and part_color are almost unique per row
# 10 car models have no working part (but there are 877 different car models)
# some days appeart to have a rather big defect rate (e.g. 3/30/2021 has 17/22 defects)

________product_id_______
ALL
99a38e48-a9bc-4348-af0a-0cd19ae6f11f    1
5d6cca75-e377-42e9-98c0-6ab4ff179044    1
51cc9119-5ebd-41a6-a6e8-a2395eb6ca95    1
461dfc13-77f2-4c78-9b6e-be9706fb5cfe    1
5c8e658a-0807-4413-bec8-f7755bc09793    1
                                       ..
fdd96cb9-0c03-473b-b4ec-87e045de9b9a    1
29332135-e7ca-4a95-981e-9dedf08dea3c    1
97efe7bd-e23a-44d2-907c-6a28835a7781    1
641aba93-cc73-4eaa-9506-bd41a8632466    1
3ffc1395-bb4e-4dc4-9ed4-5bb642123916    1
Name: product_id, Length: 5000, dtype: int64
DEFECT
4d58d555-7d84-41ad-a451-144a4afb5092    1
255c3a55-56a4-40de-b9ce-1673030ec4b8    1
ef92e2ac-586d-41a8-8889-b98042376458    1
dd3bf397-9534-439f-9e20-a40a7f71684e    1
0b8bf651-476f-405b-a1d9-08fb8aba8f88    1
                                       ..
12d30ff0-97de-4c89-ab87-b107da8aa3b9    1
f3186c1d-07ad-406a-aa67-e1a9d8b70743    1
2dd0891c-c99b-4fd8-a96e-e7d2e60756e6    1
2edef811-3d6c-449b-9a55-5e5ad6cc4ec7    1
cf65680c-e2a9-49cb-b693-0eb82d892855

In [ ]:
for column in ['production_date', 'car_model', 'car_color', 'part_material', 'plant_location']:
    all_count = df[column].value_counts()
    defect_count = df_defect[column].value_counts()
    working_count = df_working[column].value_counts()

    res = DataFrame({'all': all_count, 'defect': defect_count, 'working': working_count})
    res.fillna(0, inplace=True)
    res['defect%'] = res['defect'] / res['all']
    res.sort_values(by='defect%', ascending=False, inplace = True)
    print(f'____{column}____')
    print(res.describe())
    print(res.head())

____production_date____
              all      defect     working     defect%
count  146.000000  146.000000  146.000000  146.000000
mean    34.246575    1.938356   32.308219    0.059235
std      5.580213    2.826531    6.298949    0.103355
min     21.000000    0.000000    5.000000    0.000000
25%     31.000000    0.000000   29.000000    0.000000
50%     34.000000    1.000000   32.500000    0.041667
75%     38.000000    2.000000   37.000000    0.073848
max     48.000000   23.000000   43.000000    0.772727
           all  defect  working   defect%
3/30/2021   22    17.0        5  0.772727
1/30/2021   32    23.0        9  0.718750
5/30/2021   23    15.0        8  0.652174
2/27/2021   33     6.0       27  0.181818
3/19/2021   45     7.0       38  0.155556
____car_model____
              all      defect     working     defect%
count  877.000000  877.000000  877.000000  877.000000
mean     5.701254    0.322691    5.378563    0.054433
std      5.135574    0.654871    4.858772    0.138685
min 

In [ ]:
for column in ['temperature', 'humidity', 'pressure']:
    print(f'____{column}___')
    print(f'ALL: Min: {df[column].min()}; Max: {df[column].max()}, Avg: {df[column].mean()}, Std: {df[column].std()}')
    print(f'Defect: Min: {df_defect[column].min()}; Max: {df_defect[column].max()}, Avg: {df_defect[column].mean()}, Std: {df_defect[column].std()}')
    print(f'Working: Min: {df_working[column].min()}; Max: {df_working[column].max()}, Avg: {df_working[column].mean()}, Std: {df_working[column].std()}')
    # defects could be caused by high temperatures and low humidity -> investigate suspicious dates if these values were bad for a plant

____temperature___
ALL: Min: 1.0; Max: 120.0, Avg: 60.633161476593116, Std: 33.997969410201996
Defect: Min: 2.0; Max: 119.8, Avg: 89.46415770609319, Std: 31.622055501850124
Working: Min: 1.0; Max: 120.0, Avg: 58.87301969365427, Std: 33.34283347875909
____humidity___
ALL: Min: 1.02; Max: 99.95, Avg: 50.35313255523135, Std: 28.449297980077546
Defect: Min: 1.15; Max: 98.2, Avg: 31.66755395683454, Std: 24.16815253351732
Working: Min: 1.02; Max: 99.95, Avg: 51.50237831858407, Std: 28.29398190704208
____pressure___
ALL: Min: 1.0; Max: 1.9996, Avg: 1.499015014887282, Std: 0.2877881839061326
Defect: Min: 1.0013; Max: 1.9935, Avg: 1.4857342007434946, Std: 0.27821966306409196
Working: Min: 1.0; Max: 1.9996, Avg: 1.4998209113467176, Std: 0.28836931927413456


In [ ]:
# 

df_suspicious_days = df.loc[df['production_date'].isin(['3/30/2021', '5/30/2021', '1/30/2021'])]
df_suspicious_days.head()

,product_id,production_date,car_model,car_color,part_id,part_color,part_material,plant_location,temperature,humidity,pressure,buyer_id,first_name,last_name,email,city,age,gender,defect
30,4d58d555-7d84-41ad-a451-144a4afb5092,1/30/2021,Tracker,Yellow,275110745,#5e942a,Stone,China,76.7,NaN,1.6959,440.0,Berne,Gallear,bgallearc7@zimbio.com,Narukan,35,M,True
35,faed7b05-ef62-471b-b1b5-3817f1e96f10,1/30/2021,Edge,Violet,5910030981,#e4188e,Brass,China,56.9,17.64,1.4196,504.0,Kessia,Gowing,kgowingdz@icio.us,Pankshin,72,None,True
76,d85ea582-9c9d-4f97-be22-9320f518491a,1/30/2021,612 Scaglietti,Maroon,5427739025,#a1c635,Brass,China,62.1,83.72,1.7639,1788.0,Worthington,Moscone,wmosconelv@dagondesign.com,Dongshi,57,M,True
98,a2f65857-5db8-41f7-87fb-a0b9862f7368,1/30/2021,GTO,Pink,1543173764,#26f2e7,Plastic,Portugal,38.4,12.46,1.8609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
170,ea7e7d53-91e9-4599-8cb4-019424356f82,1/30/2021,Freelander,Puce,6998287437,#873f03,Granite,Portugal,115.9,71.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [ ]:
df_suspicious_days_defect = df_suspicious_days.loc[df_suspicious_days['defect'] == True]
df_suspicious_days_working = df_suspicious_days.loc[df_suspicious_days['defect'] == False]


all_count = df_suspicious_days['plant_location'].value_counts()
defect_count = df_suspicious_days_defect['plant_location'].value_counts()
working_count = df_suspicious_days_working['plant_location'].value_counts()

res_suspicious = DataFrame({'all': all_count, 'defect': defect_count, 'working': working_count})
res_suspicious.head()

# 100% defect rate in China

,all,defect,working
China,54,54.0,NaN
Mexico,5,NaN,5.0
Portugal,14,1.0,13.0
United States,4,NaN,4.0


In [ ]:
df_sus_days_china = df_suspicious_days.loc[df_suspicious_days['plant_location'] == 'China']

df_sus_days_china.describe()

# no indication of extreme circumstances during sus days in china

,part_id,temperature,humidity,pressure
count,5.400000e+01,52.000000,53.000000,52.000000
mean,5.333466e+09,63.005769,51.720755,1.518138
std,2.982117e+09,38.651984,29.861561,0.284059
min,1.057090e+08,4.600000,1.150000,1.002200
25%,2.878289e+09,28.600000,19.750000,1.315000
50%,5.706084e+09,61.350000,54.430000,1.529750
75%,7.641064e+09,103.375000,79.350000,1.764600
max,9.893796e+09,118.100000,94.190000,1.984700


In [ ]:
df_def_no_china_no_sus_days = df_defect.loc[~((df_defect['plant_location'] == 'China') & (df_defect['production_date'].isin(['3/30/2021', '5/30/2021', '1/30/2021'])))]

df_def_no_china_no_sus_days.describe()

# high temperatures and low humidity appear to be an issue too as suspected before

,part_id,temperature,humidity,pressure
count,2.290000e+02,227.000000,225.000000,217.000000
mean,4.774687e+09,95.525110,26.943911,1.477969
std,3.059154e+09,26.361448,19.943358,0.276904
min,3.499684e+07,2.000000,1.670000,1.001300
25%,2.111978e+09,92.150000,12.690000,1.259600
50%,4.467726e+09,102.800000,22.190000,1.491900
75%,7.790362e+09,112.900000,34.680000,1.696300
max,9.983020e+09,119.800000,98.200000,1.993500


In [ ]:
plants["part_color"].value_counts()

# part_color is distinct -> have to group "nearby colors"

#f47874    2
#a7eeff    1
#d85d7f    1
#325043    1
#a2b4c8    1
          ..
#572204    1
#9e66ca    1
#abdb84    1
#19c9fe    1
#98081f    1
Name: part_color, Length: 4999, dtype: int64

In [ ]:
from matplotlib.colors import to_rgb, to_hex

plants["part_color_rgb"] = plants["part_color"].apply(to_rgb)
plants["part_color_grp"] = plants["part_color_rgb"].apply(lambda x: (round(x[0]*2, 0)/2, round(x[1]*2, 0)/2, round(x[2]*2, 0)/2))
plants["part_color_grp"] = plants["part_color_grp"].apply(to_hex)
color_grp_dist = plants["part_color_grp"].value_counts(normalize = True)
color_grp_dist_df = color_grp_dist.to_frame().reset_index()
color_grp_dist_df = color_grp_dist_df.rename(columns={'part_color_grp': 'part_color_grp%', 'index': 'part_color_grp'})
color_grp_dist_df.head()

,part_color_grp,part_color_grp%
0,#808080,0.1300
1,#808000,0.0678
2,#80ff80,0.0668
3,#8080ff,0.0638
4,#ff8080,0.0590


In [ ]:
plants["car_model"].describe()

count         5000
unique         877
top       Corvette
freq            37
Name: car_model, dtype: object

In [ ]:
filtered = plants[["car_model", "part_color_grp"]].groupby(["car_model"]).filter(lambda x: len(x) > 9)
filtered.head()

,car_model,part_color_grp
0,Intrepid,#80ffff
3,ES,#80ff00
4,Yukon,#8000ff
5,Pathfinder,#008080
8,Justy,#ff0080


In [ ]:
grouped_color_model = filtered.groupby(["car_model", "part_color_grp"]).size().to_frame(name = 'color_model_combination_count').reset_index()
grouped_model = filtered.groupby(["car_model"]).size().to_frame(name = 'car_model_count').reset_index()


grouped = merge(left=grouped_color_model, right=grouped_model, how='left', on='car_model')

color_model_df = merge(left=grouped, right=color_grp_dist_df, how="left", on="part_color_grp")
color_model_df["car_model_color%"] = color_model_df["color_model_combination_count"]/color_model_df["car_model_count"]
color_model_df["relation"] = color_model_df["car_model_color%"]/color_model_df["part_color_grp%"]


color_model_df_significant = color_model_df.loc[(color_model_df['relation'] > 10.0) | (color_model_df['relation'] < 0.1)]
print(len(color_model_df_significant.index))
color_model_df_significant.head(10)

# there are part_color_model combination that are significant more likely according to the data but color grouping could be improved and sample size is too low

10


,car_model,part_color_grp,color_model_combination_count,car_model_count,part_color_grp%,car_model_color%,relation
78,9-5,#ff0000,2,10,0.0158,0.200000,12.658228
259,CL-Class,#00ff00,2,14,0.0116,0.142857,12.315271
510,Econoline E350,#ffffff,2,10,0.0158,0.200000,12.658228
638,F250,#ff00ff,2,11,0.0162,0.181818,11.223345
809,I,#ffffff,2,10,0.0158,0.200000,12.658228
864,LX,#ff0000,2,11,0.0158,0.181818,11.507480
933,MPV,#000000,2,12,0.0132,0.166667,12.626263
941,MPV,#ffff00,2,12,0.0138,0.166667,12.077295
981,Mirage,#000000,2,10,0.0132,0.200000,15.151515
1250,S2000,#00ffff,3,11,0.0154,0.272727,17.709563


In [ ]:
# relation between part color and car color can be calculated the same way but sample size is way higher since there are only 19 distinct car_colors
plants["car_color"].describe()

count       5000
unique        19
top       Purple
freq         287
Name: car_color, dtype: object